In [1]:
import sys
import tensorflow as tf
sys.path.append("/visuworks/Blindless_AIFFELTON/Models")

# import customized modules
import preprocess, data_generator, check_result, metrics, loss, train

sys.path.append("/visuworks/Blindless_AIFFELTON/Models/SD_Unet")
import sd_unet

# Insert Path
# {model_name}_{epoch}ep_{model_inpusize}_{Generator | aG/fG/sG}
FILE_NAME = 'sd_unet_40ep_256_aG'
INPUT_SHAPE = (256, 256, 1)
MODEL_PATH = "/visuworks/Blindless_AIFFELTON/Models/SD_Unet/model_parameters/" + FILE_NAME + ".h5"
HISTORY_PATH = "/visuworks/Blindless_AIFFELTON/Models/SD_Unet/history/" + FILE_NAME + '_history' ".json"
SOURCE = '/visuworks/Dataset/Selected Dataset 2'

2024-01-26 15:04:10.184473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 15:04:10.184500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 15:04:10.185372: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 15:04:10.190369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 15:04:10.753740: W tensorflow/compiler/tf2

In [4]:
# Define custom objects for loading the model
custom_objects = {'DiceLoss': loss.DiceLoss(), 
                  'sensitivity': metrics.sensitivity,
                  'specificity': metrics.specificity,
                  'accuracy' : metrics.accuracy,
                  'DropBlock2D': sd_unet.DropBlock2D}

# Create augmentation
train_preproc = preprocess.build_augmentation_for_adaptive()
test_preproc = preprocess.build_augmentation_for_adaptive(is_train=False)

# Create train, test generator
train_generator = data_generator.AdaptiveDataGenerator(
    SOURCE, 
    number_of_images=4,
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    is_train=True,
    augmentation= train_preproc
)

test_generator = data_generator.AdaptiveDataGenerator(
    SOURCE, 
    number_of_images=4,
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    is_train=False,
    augmentation= test_preproc
)

# Create model
model = sd_unet.SD_UNet(INPUT_SHAPE)
model.summary()

2024-01-26 15:05:32.075517: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-26 15:05:32.109036: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-26 15:05:32.109266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 8)          80        ['input_1[0][0]']             
                                                                                                  
 drop_block2d (DropBlock2D)  (None, 256, 256, 8)          131072    ['conv2d[0][0]']              
                                                                                                  
 activation (Activation)     (None, 256, 256, 8)          0         ['drop_block2d[0][0]']        
                                                                                              

In [3]:
train.model_train(model,
                  epoch = 40,
                  train_generator = train_generator,
                  test_generator = test_generator,
                  model_path = MODEL_PATH,
                  history_path = HISTORY_PATH)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: UserWarning: `tf.keras.backend.random_binomial` is deprecated, and will be removed in a future version.Please use `tf.keras.backend.random_bernoulli` instead.
  return dispatch_target(*args, **kwargs)
2024-01-26 12:50:53.089470: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-26 12:50:54.812119: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3a58815490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-26 12:50:54.812138: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-01-26 12:50:54.815478: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706273454.888273  586039 device_compiler.h:186] Compil

295/295 [==============================] - ETA: 0s - loss: 0.7308 - sensitivity: 0.0021 - specificity: 0.9964 - accuracy: 0.9302
Epoch 1: val_sensitivity improved from -inf to 0.00000, saving model to /visuworks/Blindless_AIFFELTON/Models/SD_Unet/model_parameters/sd_unet_40ep_256_aG.h5
295/295 [==============================] - 193s 618ms/step - loss: 0.7308 - sensitivity: 0.0021 - specificity: 0.9964 - accuracy: 0.9302 - val_loss: 0.6600 - val_sensitivity: 0.0000e+00 - val_specificity: 0.9993 - val_accuracy: 0.9344
Epoch 2/40


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


295/295 [==============================] - ETA: 0s - loss: 0.5801 - sensitivity: 0.2767 - specificity: 0.9725 - accuracy: 0.9267
Epoch 2: val_sensitivity improved from 0.00000 to 0.29527, saving model to /visuworks/Blindless_AIFFELTON/Models/SD_Unet/model_parameters/sd_unet_40ep_256_aG.h5
295/295 [==============================] - 186s 631ms/step - loss: 0.5801 - sensitivity: 0.2767 - specificity: 0.9725 - accuracy: 0.9267 - val_loss: 0.5636 - val_sensitivity: 0.2953 - val_specificity: 0.9905 - val_accuracy: 0.9445
Epoch 3/40
295/295 [==============================] - ETA: 0s - loss: 0.4522 - sensitivity: 0.5243 - specificity: 0.9606 - accuracy: 0.9319
Epoch 3: val_sensitivity improved from 0.29527 to 0.35474, saving model to /visuworks/Blindless_AIFFELTON/Models/SD_Unet/model_parameters/sd_unet_40ep_256_aG.h5
295/295 [==============================] - 189s 641ms/step - loss: 0.4522 - sensitivity: 0.5243 - specificity: 0.9606 - accuracy: 0.9319 - val_loss: 0.5063 - val_sensitivity: 0.3

In [5]:
# Load the model with custom_objects parameter
model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects)

# Plot history and pring evaluation of test dataset
metrics.print_evaluation(model, test_generator)
check_result.plot_history(HISTORY_PATH)

2024-01-26 15:05:47.186174: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


109/125 [=========================>....] - ETA: 6s - loss: 0.2902 - sensitivity: 0.6545 - specificity: 0.9799 - accuracy: 0.9603

In [ ]:
# Load the model with custom_objects parameter
model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects)

# Plot model result
num_images_to_select = 40
check_result.visualize_aG_result(model, num_images_to_select, SOURCE, INPUT_SHAPE, test_preproc)